* selenium 안쓰고 좋아요 크롤링 하려 했는데 안되길래 어쩔 수 없이 사용. 
* 문제 : 속도가 많이 느려짐.....약 5~6배?

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 멜론 TOP 100 차트 페이지 URL
url = 'https://www.melon.com/chart/index.htm'

# Selenium 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 헤드리스 모드(브라우저 창을 열지 않음)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 웹 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# 곡 ID, 제목, 아티스트 추출
song_ids = []
song_titles = []
song_artists = []

for song_tag in driver.find_elements(By.CSS_SELECTOR, 'tr.lst50, tr.lst100'):
    song_id = song_tag.get_attribute('data-song-no')
    if song_id:
        title_tag = song_tag.find_element(By.CSS_SELECTOR, 'div.rank01 a')
        artist_tags = song_tag.find_elements(By.CSS_SELECTOR, 'div.rank02 a')
        song_ids.append(song_id)
        song_titles.append(title_tag.text)
        song_artists.append(', '.join([artist.text for artist in artist_tags]))

driver.quit()

# 곡 가사 및 좋아요 수 추출 함수
def get_song_details(song_id):
    local_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        local_driver.get(song_url)
        time.sleep(1)  # 페이지 로드 대기
        
        # 가사 추출
        lyrics_tag = local_driver.find_element(By.CSS_SELECTOR, 'div.lyric')
        lyrics = lyrics_tag.text.replace(',', '') if lyrics_tag else "No lyrics found"

        # 좋아요 수 추출
        like_button = local_driver.find_element(By.CSS_SELECTOR, 'button.like')
        like_tag = like_button.find_element(By.CSS_SELECTOR, 'span.cnt')
        likes = like_tag.text.replace('총건수', '').strip().replace(',', '') if like_tag else '0'
        
        return song_id, lyrics, likes
    except Exception as e:
        print(f"Error fetching details for song ID {song_id}: {e}")
        return song_id, "No lyrics found", "0"
    finally:
        local_driver.quit()

# 병렬처리를 위해 ThreadPoolExecutor 사용
with ThreadPoolExecutor(max_workers=10) as executor:  # 병렬 처리를 위한 스레드풀 생성, max_workers는 병렬화할 작업 수
    results = list(executor.map(get_song_details, song_ids))

# 결과 처리
lyrics_dict = {song_id: lyrics for song_id, lyrics, _ in results}
likes_dict = {song_id: likes for song_id, _, likes in results}
lyrics_list = [lyrics_dict[song_id] for song_id in song_ids]
likes_list = [likes_dict[song_id] for song_id in song_ids]

# 데이터프레임 생성
data = {'Song ID': song_ids, 'Title': song_titles, 'Artist': song_artists, 'Lyrics': lyrics_list, 'Likes': likes_list}
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('melon_top_100(add LIKES).csv', index=False, encoding='utf-8-sig')

print("CSV 파일이 저장되었습니다.")


CSV 파일이 저장되었습니다.


## 좋아요 순 출력

In [15]:
import pandas as pd

# CSV 파일 경로
file_path = 'melon_top_100(add LIKES).csv'

# CSV 파일 로드
data = pd.read_csv(file_path)
data['Artist'] = data['Artist'].str.replace(',', '', regex=False)

# Likes 기준으로 내림차순 정렬
sorted_data = data.sort_values(by='Likes', ascending=False)

# 제목, 가수, 좋아요 수 컬럼만 선택
sorted_titles_artists_likes = sorted_data[['Title', 'Artist', 'Likes']]

# 인덱스 재설정
sorted_titles_artists_likes.reset_index(drop=True, inplace=True)

# 상위 100개 출력
print(sorted_titles_artists_likes.head(100))


                                    Title         Artist   Likes
0                                      봄날         방탄소년단   563385
1                                Dynamite         방탄소년단   464049
2   모든 날, 모든 순간 (Every day, Every Moment)            폴킴   441490
3               어떻게 이별까지 사랑하겠어, 널 사랑하는 거지     AKMU (악뮤)   439932
4                            주저하는 연인들을 위해           잔나비   421776
..                                    ...            ...     ...
95                        Merry PLLIstmas         PLAVE    22949
96                                 슬픈 초대장      순순희 (지환)    21814
97                                  꿈결같아서  민니 ((여자)아이들)    20554
98                      청혼하지 않을 이유를 못 찾았어           이무진    19668
99                                   그랬나봐    유회승 (엔플라잉)    18965

[100 rows x 3 columns]


## 가수 당 몇 곡 있는지

In [12]:
import pandas as pd

# CSV 파일 경로
file_path = 'melon_top_100(add LIKES).csv'

# CSV 파일 로드
data = pd.read_csv(file_path)

# 가수 이름에서 ','를 제거
data['Artist'] = data['Artist'].str.replace(',', '', regex=False)

# 각 가수별 곡 수를 계산
artist_song_counts = data['Artist'].value_counts()

# 결과 출력
print(artist_song_counts)


Artist
임영웅                         14
NewJeans                     8
PLAVE                        6
IVE (아이브)                    5
RIIZE                        4
aespa                        4
세븐틴 (SEVENTEEN)              3
DAY6 (데이식스)                  3
LE SSERAFIM (르세라핌)           3
아일릿(ILLIT)                   2
방탄소년단                        2
(여자)아이들                      2
태연 (TAEYEON)                 2
이무진                          2
아이유                          2
이클립스 (ECLIPSE)               2
AKMU (악뮤)                    2
잔나비                          1
우디 (Woody)                   1
VIVIZ (비비지)                  1
부석순 (SEVENTEEN)              1
민니 ((여자)아이들)                 1
폴킴                           1
로이킴                          1
윤하 (YOUNHA)                  1
김민석                          1
임한별                          1
경서예지 전건호                     1
Charlie Puth                 1
순순희(기태)                      1
DK(디셈버)                      1
성시경                          1
순